In [7]:
from data.image_pair_dataset import Paired_Dataset
from torchvision.transforms import transforms


data_dirs = ['./training_data/DIV2K_train_HR', './training_data/Flickr2K']
data = Paired_Dataset(image_directories=data_dirs, scale=2, patch_size=(500, 500))

hr, lr = data[470]
print(hr.shape, lr.shape)

to_pil = transforms.ToPILImage()

to_pil(hr).show()
to_pil(lr).show()

torch.Size([3, 1000, 1000]) torch.Size([3, 500, 500])


In [4]:
from data.get_loader import get_loader
from torchvision.transforms import transforms

data_dirs = ['./training_data/DIV2K_train_HR', './training_data/Flickr2K']
prefetcher = get_loader(image_directories=data_dirs,
                        scale=4,
                        patch_size=(100, 100),
                        batch_size=1,
                        device=None,
                        num_workers=2,
                        pin_memory=False)


batch = prefetcher.next()
lr, hr = batch
print(lr.shape, hr.shape)
lr = lr.squeeze(0)
hr = hr.squeeze(0)
print(lr.shape, hr.shape)

to_pil = transforms.ToPILImage()

to_pil(hr).show()
to_pil(lr).show()

torch.Size([1, 3, 100, 100]) torch.Size([1, 3, 400, 400])
torch.Size([3, 100, 100]) torch.Size([3, 400, 400])


In [14]:
from train.trainer import Trainer

trainer = Trainer('train_yamls/RWVKSR_2X.yaml')
trainer.train()

config loaded successfully


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


loss,▇▅█▇▅▅▅▆▄▅▂▃▃▅▅█▆▁▄▅▆▄▄▅▆▆▇▆▄▆▃▄▃▄▃▅▃▅▆▃
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.01793
lr,0.001


model has 304140 trainable parameters
training on mps


Epoch 20/20: 100%|██████████| 108/108 [01:23<00:00,  1.29it/s, loss=0.0191, lr=0.0010]


In [16]:
import torch
from torchvision.transforms import v2 as T
from torchvision.io import read_image
from arch.model import RWKVSR

state_dict = torch.load('checkpoints/2X_RWKVSR/epoch_15.pt')
epoch = state_dict['epoch']
model = RWKVSR(2, 3, 64, 4)
model.load_state_dict(state_dict['model_state_dict'])
model.eval()

img = read_image('training_data/test.png')
img = T.ToDtype(torch.float32, scale=True)(img)

pred = model(img)
pred = torch.clamp(pred, 0, 1)

to_pil = T.ToPILImage()
final = to_pil(pred)
final.save(f'test_outputs/epoch_{epoch}.png')

print(f'epoch evaluated: {epoch}')


epoch evaluated: 15
